In [28]:
import cv2
import sys
import struct
import imageio
import numpy as np
from libtiff import TIFF

def ReadTiff(filename):
#     img = imageio.imread(filename)
    img = np.array(cv2.imread(filename, cv2.IMREAD_UNCHANGED))
    return img

def tiff_to_image_array(tiff_image_name): 
          
    tif = TIFF.open(tiff_image_name, mode = "r")
    res = []
    for im in list(tif.iter_images()):
        res.append(im)
    return res

def header(shape, bits):
    res = []
    for i in shape:
        res.append(uint32_to_byte(i))
    res.append(uint32_to_byte(bits))
    return res

def uint32_to_byte(value):
    return int.to_bytes(value, 4, byteorder='big', signed=False)

def uint16_to_byte(value):
    return int.to_bytes(value, 2, byteorder='big', signed=False)

def uint8_to_byte(value):
    return int.to_bytes(value, 1, byteorder='big', signed=False)

def bytes_to_int(value):
    return int.from_bytes(value, byteorder='big', signed=False)



In [ ]:
# np.linspace仅是为了方便调试生成的测试数据，生成正式的图像文件可去掉第一行注释
# img = np.array(tiff_to_image_array('/home/zwj/Desktop/120tp_2_f1_c1.tif'))
img = np.linspace(1, 72, 72).reshape(3,4,6)

# 图像解构
img = img.transpose((1,2,0))
print(img.shape)
# print(img[:,:,2])
shape = img.shape
head_info = header(shape, 16)
img_bytes = []
for k in range(img.shape[2]):
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            img_bytes.append(uint16_to_byte(int(img[i,j,k])))

with open('/home/zwj/Desktop/img.bin', 'wb') as f:
    for i in head_info:
        f.write(i)
    for i in img_bytes:
        f.write(i)
print('Done.')

In [31]:
import numpy as np

# 图像重构，便于显示
with open('/home/zwj/IntellijProjects/BigData/img.bin', 'rb') as f:
    data = f.read()
    x, y, z = bytes_to_int(data[0:4]), bytes_to_int(data[4:8]), bytes_to_int(data[8:12])
    offset = bytes_to_int(data[12:16]) // 8
    data = data[16:]
    img = np.zeros((x,y,z)).astype(np.uint16)
    length = 0
    for k in range(img.shape[2]):
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                img[i,j,k] = bytes_to_int(data[length : length + offset])
                length += offset
    
    srcimg = np.linspace(1, 72, 72).reshape(3,4,6)
    srcimg = srcimg.transpose((1,2,0))
    print((srcimg == img).sum())

72
